In [1]:
%store -r BER_Dublin_specific

In [2]:
import pandas as pd
import numpy as np

In [3]:
# SH Demand = Main SH + Sec SH

def calc_main_SH_Demand(
    CHPSystemType,
    gsdHSSupplHeatFraction,
    delivered_energy_main,
    effficiency,
    efficiency_adjustment_factor,
    CHPUnitHeatFraction,
):
    eff_adj = effficiency * efficiency_adjustment_factor
    
    # Group Heating (i.e. HH uses CHP or waste energy for HW & SH)
    """ EXCEL FORMULA: (1-K6)*ShReqt/H12
        K6 = Fraction of heat use from secondary / supplementary system (from Table 7, Table 10 or Appendix F)
        ShReqt = Annual space heating requirement [kWh/y] 
               = Net heat emission to heated space [kWh/y] + Additional heat loss [kWh/y]
        H12 = Efficiency factor for charging method [-] = 0.9
        """
    if pd.notnull(CHPSystemType) :
        
        return delivered_energy_main * 0.9
            
    # Individual Heating 
    """ EXCEL FORMULA: IF(J7=0,0,(1-M21)*F125/(J7/100))
        J7 = Adjusted efficiency of main heating system [%]
        M21 = Fraction of main space and water heat from CHP
        F125 = Total heat demand main space heating [kWh/y]
        """
    if CHPUnitHeatFraction < 1:
        return (1 / 1 - CHPUnitHeatFraction) * delivered_energy_main * (eff_adj / 100) 
    else:
        return 0

calc_main_SH_Demand = np.vectorize(calc_main_SH_Demand)

In [4]:
BER_Dublin_specific[r"DeliveredEnergyMainSpace (ind)"] = calc_main_SH_Demand(
    BER_Dublin_specific["CHPSystemType"].values,
    BER_Dublin_specific["gsdHSSupplHeatFraction_edited"].values,
    BER_Dublin_specific["DeliveredEnergyMainSpace"].values,
    BER_Dublin_specific["HSMainSystemEfficiency"].values,
    BER_Dublin_specific["HSEffAdjFactor"].values,
    BER_Dublin_specific["CHPUnitHeatFraction"].values,
    
)

In [5]:
BER_Dublin_specific[
    [
        "DeliveredEnergyMainSpace",
        "DeliveredEnergyMainSpace (ind)",
        "HSMainSystemEfficiency",
    ]
].head()

,DeliveredEnergyMainSpace,DeliveredEnergyMainSpace (ind),HSMainSystemEfficiency
2,10826.731,9287.711188,90.3
6,20046.920,13712.093280,72.0
15,14756.551,9112.170243,65.0
18,11019.516,6804.551130,65.0
22,8743.726,6504.020585,78.3


In [6]:
%store BER_Dublin_specific

Stored 'BER_Dublin_specific' (DataFrame)
